In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn import model as modellib, utils
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
import imgaug
import imgaug.augmenters as iaa
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
def get_ax(rows=1, cols=1, size=16):
  """Return a Matplotlib Axes array to be used in all visualizations in the notebook.  Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\framework\dtypes.py:5

In [2]:
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options)) # allocate memory for gpu
tf.device('/gpu:0') # check for gpu
Root_Dir=r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024" #main directory where all files are present
sys.path.append(Root_Dir)
COCO_Weights_path=os.path.join(Root_Dir,"mask_rcnn_coco.h5") # pretrained weights for mask rcnn used to initialize the network
Default_Logs_Dir=os.path.join(Root_Dir,"logs")

# config of mask rcnn, hyperparameters set here
class CustomConfig(Config):
    NAME="object"
    IMAGES_PER_GPU=1
    NUM_CLASSES=32+1
    BACKBONE ='resnet50'
    STEPS_PER_EPOCH=907/(IMAGES_PER_GPU)
    VALIDATION_STEPS=261
    ROI_POSITIVE_RATIO = 0.334
    DETECTION_MIN_CONFIDENCE=0.7
    IMAGE_SHAPE=[1024,1024,3]
    IMAGE_MAX_DIM=1024
    LEARNING_RATE=0.001
    USE_MINI_MASK=True # AVOID DOWNSAMPLING to improve accuarcy. check if accuracy improves
    TRAIN_ROIS_PER_IMAGE=200 # REDUCE rois from 200 to 100 to improve runtime
    MAX_GT_INSTANCES=100 # reduce it from 100 to 50 to improve runtime
    #POST_NMS_ROIS_INFERENCE=500 #halving the vaue to reduce runtime
    #POST_NMS_ROIS_TRAINING=1000 # having the value to reduce runtime
    #PRE_NMS_LIMIT=3000 # halving the value to reduce runtime
    
    
    
config=CustomConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                45
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

In [3]:
#config=CustomConfig()
#customcallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50,restore_best_weights=True)

In [4]:
#initialize dataset loader


class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir,file, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have 32 classes to add.
        self.add_class("object",1,"1")
        self.add_class("object",2,"2")
        self.add_class("object",3,"3")
        self.add_class("object",4,"4")
        self.add_class("object",5,"5")
        self.add_class("object",6,"6")
        self.add_class("object",7,"7")
        self.add_class("object",8,"8")
        self.add_class("object",9,"9")
        self.add_class("object",10,"10")
        self.add_class("object",11,"11")
        self.add_class("object",12,"12")
        self.add_class("object",13,"13")
        self.add_class("object",14,"14")
        self.add_class("object",15,"15")
        self.add_class("object",16,"16")
        self.add_class("object",17,"17")
        self.add_class("object",18,"18")
        self.add_class("object",19,"19")
        self.add_class("object",20,"20")
        self.add_class("object",21,"21")
        self.add_class("object",22,"22")
        self.add_class("object",23,"23")
        self.add_class("object",24,"24")
        self.add_class("object",25,"25")
        self.add_class("object",26,"26")
        self.add_class("object",27,"27")
        self.add_class("object",28,"28")
        self.add_class("object",29,"29")
        self.add_class("object",30,"30")
        self.add_class("object",31,"31")
        self.add_class("object",32,"32")
        
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations1=json.load(open(file))['_via_img_metadata']
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        counter=0
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['tooth'] for s in a['regions']]
            print("objects:",objects)
            
            name_dict = {"1": 1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"10":10,"11":11,"12":12,"13":13,"14":14,"15":15,"16":16,"17":17,"18":18,"19":19,"20":20,"21":21,"22":22,"23":23,"24":24,"25":25,"26":26,"27":27,"28":28,"29":29,"30":30,"31":31,"32":32}
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            counter=counter+1
            print(counter)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)
    def load_custom_K_fold(self, dataset_dir,file, subset, fold):
        # Add classes
        self.add_class("object",1,"1")
        self.add_class("object",2,"2")
        self.add_class("object",3,"3")
        self.add_class("object",4,"4")
        self.add_class("object",5,"5")
        self.add_class("object",6,"6")
        self.add_class("object",7,"7")
        self.add_class("object",8,"8")
        self.add_class("object",9,"9")
        self.add_class("object",10,"10")
        self.add_class("object",11,"11")
        self.add_class("object",12,"12")
        self.add_class("object",13,"13")
        self.add_class("object",14,"14")
        self.add_class("object",15,"15")
        self.add_class("object",16,"16")
        self.add_class("object",17,"17")
        self.add_class("object",18,"18")
        self.add_class("object",19,"19")
        self.add_class("object",20,"20")
        self.add_class("object",21,"21")
        self.add_class("object",22,"22")
        self.add_class("object",23,"23")
        self.add_class("object",24,"24")
        self.add_class("object",25,"25")
        self.add_class("object",26,"26")
        self.add_class("object",27,"27")
        self.add_class("object",28,"28")
        self.add_class("object",29,"29")
        self.add_class("object",30,"30")
        self.add_class("object",31,"31")
        self.add_class("object",32,"32")

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, 'train')        #split train folder to k-fold train and val

        N_Folds = 5
        
        annotations = []

        annotation =json.load(open(file))['_via_img_metadata']
        
        annotation = list(annotation.values())  # don't need the dict keys

        k_fold = KFold(n_splits = N_Folds, random_state = 42, shuffle = True)

        for i, (train, val) in enumerate(k_fold.split(annotation)):
            if subset == "train" and fold == i:
                for index in train:
                    annotations.append(annotation[index])

            elif subset == "val" and fold == i:
                for index in val:
                    annotations.append(annotation[index])

        annotations = [a for a in annotations if a['regions']]

        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['tooth'] for s in a['regions']]
            name_dict = {"1": 1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"10":10,"11":11,"12":12,"13":13,"14":14,"15":15,"16":16,"17":17,"18":18,"19":19,"20":20,"21":21,"22":22,"23":23,"24":24,"25":25,"26":26,"27":27,"28":28,"29":29,"30":30,"31":31,"32":32}
            num_ids = [name_dict[a] for a in objects]
            
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            if os.path.exists(image_path):
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]

                self.add_image(
                    "object",  ## for a single class just add the name here
                    image_id=a['filename'],  # use file name as a unique image id
                    path=image_path,
                    width=width, height=height,
                    polygons=polygons,
                    num_ids=num_ids)

In [5]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=Default_Logs_Dir)

dataset_train = CustomDataset()
dataset_train.load_custom(r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\33 class classification\dataset",
                               r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\33 class classification\dataset\train\train.json", 
                                   "train")
dataset_train.prepare() 

dataset_val = CustomDataset()
dataset_val.load_custom(r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\33 class classification\dataset",
                                           r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\33 class classification\dataset\val\val.json", 
                                               "val")
dataset_val.prepare() 

model_inference = modellib.MaskRCNN(mode="inference", model_dir=Default_Logs_Dir, config=CustomConfig()) # prepare mask rcnn for testing
mAP_callback = modellib.MeanAveragePrecisionCallback(model, model_inference, dataset_val, 
                                                        calculate_at_every_X_epoch=1, dataset_limit=500, verbose=1)
weights_path = COCO_Weights_path
if not os.path.exists(weights_path):
    utils.download_trained_weights(weights_path)

       


Instructions for updating:
Colocations handled automatically by placer.
objects: ['2', '3', '32', '31', '30', '17', '18', '15', '14', '19', '4', '5', '6', '29', '28', '27', '26', '25', '24', '23', '22', '21', '20', '13', '12', '11']
numids [2, 3, 32, 31, 30, 17, 18, 15, 14, 19, 4, 5, 6, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 13, 12, 11]
1
objects: ['2', '31', '15', '18', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
numids [2, 31, 15, 18, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
2
objects: ['2', '3', '32', '31', '4', '30', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
numids [2, 3, 32, 31, 4, 30, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
3
objects: ['2', '3', '31', '30', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '19'

objects: ['1', '2', '3', '5', '6', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '24', '24', '25', '26', '27', '28', '29', '30', '31', '32']
numids [1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 24, 25, 26, 27, 28, 29, 30, 31, 32]
36
objects: ['1', '2', '4', '6', '15', '14', '13', '12', '11', '10', '9', '8', '7', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [1, 2, 4, 6, 15, 14, 13, 12, 11, 10, 9, 8, 7, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
37
objects: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
numids [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
38
objects: ['1', '2', '3', '5', '6', '7', '8', '9', '10', '11', '12', '13', '15', '17', '18', '

objects: ['2', '3', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '32', '31', '29', '28', '27', '26', '25', '24', '23', '22', '21', '20', '18', '17']
numids [2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 32, 31, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 18, 17]
66
objects: ['2', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '18', '29', '28', '27', '26', '25', '24', '23', '22', '21']
numids [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 18, 29, 28, 27, 26, 25, 24, 23, 22, 21]
67
objects: ['1', '2', '3', '4', '6', '7', '8', '9', '10', '11', '12', '14', '15', '16', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '5', '13']
numids [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 5, 13]
68
objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31

objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
95
objects: ['1', '2', '3', '4', '6', '7', '8', '9', '10', '11', '12', '14', '15', '13', '17', '20', '21', '23', '24', '25', '26', '27', '28', '29', '31', '32']
numids [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 14, 15, 13, 17, 20, 21, 23, 24, 25, 26, 27, 28, 29, 31, 32]
96
objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '23']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 23]
97
objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '18', '19', '20', '21', '22', '23', '24', '27', '28', '29', '31']
numids [2, 3, 4, 5,

objects: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
numids [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
130
objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
131
objects: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
numids [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
132
objects: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',

...

objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
130
objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
131
objects: ['1', '2', '3', '4', '5', '6', '8', '9', '10', '12', '13', '14', '15', '16', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
numids [1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
132
objects: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19

objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
159
objects: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
numids [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
160
objects: ['5', '6', '11', '21', '23', '24', '25', '26', '27', '28', '29', '22']
numids [5, 6, 11, 21, 23, 24, 25, 26, 27, 28, 29, 22]
161
objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24

objects: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
193
objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
194
objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
195
objects: ['1', '2', '4', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '

objects: ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
numids [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
225
objects: ['2', '4', '6', '7', '8', '9', '10', '11', '12', '15', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
numids [2, 4, 6, 7, 8, 9, 10, 11, 12, 15, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
226
objects: ['21', '22', '23', '24', '25', '26', '27', '28']
numids [21, 22, 23, 24, 25, 26, 27, 28]
227
objects: ['2', '3', '5', '6', '7', '8', '9', '11', '12', '13', '14', '15', '17', '18', '20', '21', '22', '24', '25', '26', '27', '28', '29', '30']
numids [2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30]
228
objects: ['5', '6', '8', '11', '12', '22', '23', '24', '25', '26', '27']
numids [5, 6, 8, 11, 12, 22, 23, 24, 25, 26, 27]
229
objects: ['1', '2'

objects: ['2', '3', '4', '6', '7', '8', '9', '10', '11', '12', '14', '15', '18', '19', '21', '22', '23', '24', '25', '26', '27', '28', '30', '31', '31']
numids [2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 14, 15, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 31]
259
objects: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']
numids [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
260
objects: ['2', '3', '3', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '18', '22', '23', '24', '25', '26', '27', '31']
numids [2, 3, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 22, 23, 24, 25, 26, 27, 31]
261
Instructions for updating:
Use tf.cast instead.


In [7]:
model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=20,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230521T2348\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
907/907 [==============================] - 3060s 3s/step - loss: 3.0537 - rpn_class_loss: 0.1710 - rpn_bbox_loss: 0.7426 - mrcnn_class_loss: 0.9869 - mrcnn_bbox_loss: 0.4614 - mrcnn_mask_loss: 0.6918 - val_loss: 2.2541 - val_rpn_class_loss: 0.0402 - val_rpn_bbox_loss: 0.4768 - val_mrcnn_class_loss: 0.7097 - val_mrcnn_bbox_loss: 0.3484 - val_mrcnn_mask_loss: 0.6790
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230521T2348\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 1.5606092248921453 %
Epoch 2/20
907/907 [==============================] - 3314s 4s/step - loss: 2.2008 - rpn_class_loss: 0.0715 - rpn_bbox_loss: 0.5409 - mrcnn_class_loss: 0.6321 - mrcnn_bbox_loss: 0.3200 - mrcnn_mask_loss: 0.6364 - val_loss: 1.8791 - val_rpn_class_loss: 0.0351 - val_rpn_bbox_loss: 0.4792 - val_mrcnn_class_loss: 0.5664 - val_mrcnn_bbox_loss: 

Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230521T2348\mask_rcnn_object_0013.h5
Re-starting from epoch 13
mAP at epoch 13 is: 77.38255384413442 %
Epoch 14/20
907/907 [==============================] - 3259s 4s/step - loss: 1.2510 - rpn_class_loss: 0.0228 - rpn_bbox_loss: 0.3351 - mrcnn_class_loss: 0.3441 - mrcnn_bbox_loss: 0.2456 - mrcnn_mask_loss: 0.3034 - val_loss: 1.1985 - val_rpn_class_loss: 0.0225 - val_rpn_bbox_loss: 0.3250 - val_mrcnn_class_loss: 0.3649 - val_mrcnn_bbox_loss: 0.1934 - val_mrcnn_mask_loss: 0.2927
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230521T2348\mask_rcnn_object_0014.h5
Re-starting from epoch 14
mAP at epoch 14 is: 79.3286452888807 %
Epoch 15/20
  1/907 [..............................] - ETA

KeyboardInterrupt: 

In [ ]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=10,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230523T0151\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
907/907 [==============================] - 3001s 3s/step - loss: 1.1821 - rpn_class_loss: 0.0212 - rpn_bbox_loss: 0.3269 - mrcnn_class_loss: 0.3365 - mrcnn_bbox_loss: 0.2020 - mrcnn_mask_loss: 0.2955 - val_loss: 1.1978 - val_rpn_class_loss: 0.0206 - val_rpn_bbox_loss: 0.3311 - val_mrcnn_class_loss: 0.3610 - val_mrcnn_bbox_loss: 0.1980 - val_mrcnn_mask_loss: 0.2871
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230523T0151\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 81.060026222335 %
Epoch 2/10
907/907 [==============================] - 2725s 3s/step - loss: 1.1771 - rpn_class_loss: 0.0230 - rpn_bbox_loss: 0.3283 - mrcnn_class_loss: 0.3264 - mrcnn_bbox_loss: 0.1983 - mrcnn_mask_loss: 0.3010 - val_loss: 1.1730 - val_rpn_class_loss: 0.0209 - val_rpn_bbox_loss: 0.3169 - val_mrcnn_class_loss: 0.3584 - val_mrcnn_bbox_loss: 0.1

In [ ]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230524T0107\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
907/907 [==============================] - 3616s 4s/step - loss: 1.0725 - rpn_class_loss: 0.0199 - rpn_bbox_loss: 0.3053 - mrcnn_class_loss: 0.2909 - mrcnn_bbox_loss: 0.1734 - mrcnn_mask_loss: 0.2829 - val_loss: 1.1453 - val_rpn_class_loss: 0.0182 - val_rpn_bbox_loss: 0.3129 - val_mrcnn_class_loss: 0.3506 - val_mrcnn_bbox_loss: 0.1861 - val_mrcnn_mask_loss: 0.2776
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230524T0107\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 83.86008271700123 %
Epoch 2/15
907/907 [==============================] - 2709s 3s/step - loss: 1.0790 - rpn_class_loss: 0.0187 - rpn_bbox_loss: 0.3025 - mrcnn_class_loss: 0.2931 - mrcnn_bbox_loss: 0.1781 - mrcnn_mask_loss: 0.2866 - val_loss: 1.0913 - val_rpn_class_loss: 0.0185 - val_rpn_bbox_loss: 0.3015 - val_mrcnn_class_loss: 0.3265 - val_mrcnn_bbox_loss: 0

Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230524T0107\mask_rcnn_object_0013.h5
Re-starting from epoch 13
mAP at epoch 13 is: 86.35306907489743 %
Epoch 14/15
907/907 [==============================] - 3107s 3s/step - loss: 0.9763 - rpn_class_loss: 0.0167 - rpn_bbox_loss: 0.2771 - mrcnn_class_loss: 0.2640 - mrcnn_bbox_loss: 0.1453 - mrcnn_mask_loss: 0.2731 - val_loss: 1.0738 - val_rpn_class_loss: 0.0168 - val_rpn_bbox_loss: 0.2910 - val_mrcnn_class_loss: 0.3248 - val_mrcnn_bbox_loss: 0.1746 - val_mrcnn_mask_loss: 0.2666
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230524T0107\mask_rcnn_object_0014.h5
Re-starting from epoch 14
mAP at epoch 14 is: 86.45556670482112 %
Epoch 15/15
907/907 [==============================] - 38

In [ ]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230526T0059\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
907/907 [==============================] - 2973s 3s/step - loss: 0.9686 - rpn_class_loss: 0.0166 - rpn_bbox_loss: 0.2757 - mrcnn_class_loss: 0.2632 - mrcnn_bbox_loss: 0.1415 - mrcnn_mask_loss: 0.2716 - val_loss: 1.0826 - val_rpn_class_loss: 0.0180 - val_rpn_bbox_loss: 0.2946 - val_mrcnn_class_loss: 0.3308 - val_mrcnn_bbox_loss: 0.1734 - val_mrcnn_mask_loss: 0.2658
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230526T0059\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 86.74650034978977 %
Epoch 2/15
907/907 [==============================] - 2754s 3s/step - loss: 0.9609 - rpn_class_loss: 0.0161 - rpn_bbox_loss: 0.2755 - mrcnn_class_loss: 0.2562 - mrcnn_bbox_loss: 0.1406 - mrcnn_mask_loss: 0.2726 - val_loss: 1.0639 - val_rpn_class_loss: 0.0161 - val_rpn_bbox_loss: 0.2902 - val_mrcnn_class_loss: 0.3199 - val_mrcnn_bbox_loss: 0

In [6]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=6,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230526T0956\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/6
907/907 [==============================] - 3821s 4s/step - loss: 0.9317 - rpn_class_loss: 0.0147 - rpn_bbox_loss: 0.2674 - mrcnn_class_loss: 0.2483 - mrcnn_bbox_loss: 0.1333 - mrcnn_mask_loss: 0.2680 - val_loss: 1.0655 - val_rpn_class_loss: 0.0172 - val_rpn_bbox_loss: 0.2985 - val_mrcnn_class_loss: 0.3246 - val_mrcnn_bbox_loss: 0.1682 - val_mrcnn_mask_loss: 0.2570
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230526T0956\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 86.50411505951723 %
Epoch 2/6
907/907 [==============================] - 2883s 3s/step - loss: 0.9327 - rpn_class_loss: 0.0147 - rpn_bbox_loss: 0.2666 - mrcnn_class_loss: 0.2496 - mrcnn_bbox_loss: 0.1349 - mrcnn_mask_loss: 0.2668 - val_loss: 1.0579 - val_rpn_class_loss: 0.0180 - val_rpn_bbox_loss: 0.2807 - val_mrcnn_class_loss: 0.3257 - val_mrcnn_bbox_loss: 0.1

In [5]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230529T0105\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
907/907 [==============================] - 3009s 3s/step - loss: 0.9132 - rpn_class_loss: 0.0160 - rpn_bbox_loss: 0.2613 - mrcnn_class_loss: 0.2459 - mrcnn_bbox_loss: 0.1260 - mrcnn_mask_loss: 0.2639 - val_loss: 1.0671 - val_rpn_class_loss: 0.0178 - val_rpn_bbox_loss: 0.2856 - val_mrcnn_class_loss: 0.3309 - val_mrcnn_bbox_loss: 0.1679 - val_mrcnn_mask_loss: 0.2648
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230529T0105\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 86.62181344716777 %
Epoch 2/15
907/907 [==============================] - 2977s 3s/step - loss: 0.9055 - rpn_class_loss: 0.0144 - rpn_bbox_loss: 0.2620 - mrcnn_class_loss: 0.2396 - mrcnn_bbox_loss: 0.1255 - mrcnn_mask_loss: 0.2639 - val_loss: 1.0550 - val_rpn_class_loss: 0.0163 - val_rpn_bbox_loss: 0.2877 - val_mrcnn_class_loss: 0.3210 - val_mrcnn_bbox_loss: 0

Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230529T0105\mask_rcnn_object_0013.h5
Re-starting from epoch 13
mAP at epoch 13 is: 87.0970383841642 %
Epoch 14/15
907/907 [==============================] - 2981s 3s/step - loss: 0.8492 - rpn_class_loss: 0.0126 - rpn_bbox_loss: 0.2484 - mrcnn_class_loss: 0.2199 - mrcnn_bbox_loss: 0.1117 - mrcnn_mask_loss: 0.2566 - val_loss: 1.0457 - val_rpn_class_loss: 0.0181 - val_rpn_bbox_loss: 0.2800 - val_mrcnn_class_loss: 0.3232 - val_mrcnn_bbox_loss: 0.1671 - val_mrcnn_mask_loss: 0.2574
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230529T0105\mask_rcnn_object_0014.h5
Re-starting from epoch 14
mAP at epoch 14 is: 87.86473929794816 %
Epoch 15/15
907/907 [==============================] - 295

In [6]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230530T0115\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
907/907 [==============================] - 3023s 3s/step - loss: 0.8408 - rpn_class_loss: 0.0138 - rpn_bbox_loss: 0.2464 - mrcnn_class_loss: 0.2164 - mrcnn_bbox_loss: 0.1092 - mrcnn_mask_loss: 0.2551 - val_loss: 1.0458 - val_rpn_class_loss: 0.0166 - val_rpn_bbox_loss: 0.2794 - val_mrcnn_class_loss: 0.3274 - val_mrcnn_bbox_loss: 0.1665 - val_mrcnn_mask_loss: 0.2558
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230530T0115\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 88.31799542663559 %
Epoch 2/15
907/907 [==============================] - 2843s 3s/step - loss: 0.8359 - rpn_class_loss: 0.0132 - rpn_bbox_loss: 0.2427 - mrcnn_class_loss: 0.2149 - mrcnn_bbox_loss: 0.1094 - mrcnn_mask_loss: 0.2558 - val_loss: 1.0414 - val_rpn_class_loss: 0.0165 - val_rpn_bbox_loss: 0.2777 - val_mrcnn_class_loss: 0.3263 - val_mrcnn_bbox_loss: 0

Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230530T0115\mask_rcnn_object_0013.h5
Re-starting from epoch 13
mAP at epoch 13 is: 89.29092781205856 %
Epoch 14/15
907/907 [==============================] - 2847s 3s/step - loss: 0.7953 - rpn_class_loss: 0.0124 - rpn_bbox_loss: 0.2343 - mrcnn_class_loss: 0.2004 - mrcnn_bbox_loss: 0.0988 - mrcnn_mask_loss: 0.2495 - val_loss: 1.0395 - val_rpn_class_loss: 0.0173 - val_rpn_bbox_loss: 0.2888 - val_mrcnn_class_loss: 0.3120 - val_mrcnn_bbox_loss: 0.1694 - val_mrcnn_mask_loss: 0.2521
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230530T0115\mask_rcnn_object_0014.h5
Re-starting from epoch 14
mAP at epoch 14 is: 87.90098682305663 %
Epoch 15/15
907/907 [==============================] - 28

In [ ]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230531T0104\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
907/907 [==============================] - 2980s 3s/step - loss: 0.7887 - rpn_class_loss: 0.0116 - rpn_bbox_loss: 0.2334 - mrcnn_class_loss: 0.1996 - mrcnn_bbox_loss: 0.0967 - mrcnn_mask_loss: 0.2474 - val_loss: 1.0574 - val_rpn_class_loss: 0.0168 - val_rpn_bbox_loss: 0.2845 - val_mrcnn_class_loss: 0.3340 - val_mrcnn_bbox_loss: 0.1683 - val_mrcnn_mask_loss: 0.2538
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230531T0104\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 89.34651865942737 %
Epoch 2/15
907/907 [==============================] - 2691s 3s/step - loss: 0.7826 - rpn_class_loss: 0.0119 - rpn_bbox_loss: 0.2320 - mrcnn_class_loss: 0.1951 - mrcnn_bbox_loss: 0.0964 - mrcnn_mask_loss: 0.2473 - val_loss: 1.0526 - val_rpn_class_loss: 0.0153 - val_rpn_bbox_loss: 0.2757 - val_mrcnn_class_loss: 0.3344 - val_mrcnn_bbox_loss: 0

Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230531T0104\mask_rcnn_object_0013.h5
Re-starting from epoch 13
mAP at epoch 13 is: 89.44824098757942 %
Epoch 14/15
907/907 [==============================] - 2962s 3s/step - loss: 0.7473 - rpn_class_loss: 0.0112 - rpn_bbox_loss: 0.2237 - mrcnn_class_loss: 0.1806 - mrcnn_bbox_loss: 0.0894 - mrcnn_mask_loss: 0.2424 - val_loss: 1.0393 - val_rpn_class_loss: 0.0154 - val_rpn_bbox_loss: 0.2693 - val_mrcnn_class_loss: 0.3309 - val_mrcnn_bbox_loss: 0.1738 - val_mrcnn_mask_loss: 0.2498
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230531T0104\mask_rcnn_object_0014.h5
Re-starting from epoch 14
mAP at epoch 14 is: 88.59246357031535 %
Epoch 15/15
907/907 [==============================] - 29

In [6]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230601T0010\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
907/907 [==============================] - 2921s 3s/step - loss: 0.7421 - rpn_class_loss: 0.0115 - rpn_bbox_loss: 0.2211 - mrcnn_class_loss: 0.1801 - mrcnn_bbox_loss: 0.0874 - mrcnn_mask_loss: 0.2420 - val_loss: 1.0541 - val_rpn_class_loss: 0.0160 - val_rpn_bbox_loss: 0.2789 - val_mrcnn_class_loss: 0.3368 - val_mrcnn_bbox_loss: 0.1739 - val_mrcnn_mask_loss: 0.2485
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230601T0010\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 88.80943543267283 %
Epoch 2/15
907/907 [==============================] - 2854s 3s/step - loss: 0.7372 - rpn_class_loss: 0.0113 - rpn_bbox_loss: 0.2201 - mrcnn_class_loss: 0.1783 - mrcnn_bbox_loss: 0.0870 - mrcnn_mask_loss: 0.2406 - val_loss: 1.0543 - val_rpn_class_loss: 0.0163 - val_rpn_bbox_loss: 0.2867 - val_mrcnn_class_loss: 0.3341 - val_mrcnn_bbox_loss: 0

Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230601T0010\mask_rcnn_object_0013.h5
Re-starting from epoch 13
mAP at epoch 13 is: 89.13925302361669 %
Epoch 14/15
907/907 [==============================] - 2783s 3s/step - loss: 0.7104 - rpn_class_loss: 0.0105 - rpn_bbox_loss: 0.2129 - mrcnn_class_loss: 0.1688 - mrcnn_bbox_loss: 0.0811 - mrcnn_mask_loss: 0.2372 - val_loss: 1.0672 - val_rpn_class_loss: 0.0168 - val_rpn_bbox_loss: 0.2804 - val_mrcnn_class_loss: 0.3417 - val_mrcnn_bbox_loss: 0.1783 - val_mrcnn_mask_loss: 0.2500
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230601T0010\mask_rcnn_object_0014.h5
Re-starting from epoch 14
mAP at epoch 14 is: 89.30010703093647 %
Epoch 15/15
907/907 [==============================] - 28

In [6]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230602T0039\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mr

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
907/907 [==============================] - 3038s 3s/step - loss: 0.7057 - rpn_class_loss: 0.0109 - rpn_bbox_loss: 0.2125 - mrcnn_class_loss: 0.1664 - mrcnn_bbox_loss: 0.0797 - mrcnn_mask_loss: 0.2362 - val_loss: 1.0570 - val_rpn_class_loss: 0.0166 - val_rpn_bbox_loss: 0.2777 - val_mrcnn_class_loss: 0.3391 - val_mrcnn_bbox_loss: 0.1757 - val_mrcnn_mask_loss: 0.2479
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\boxed_images\10241024\logs\object20230602T0039\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 89.45733931170007 %
Epoch 2/15
907/907 [==============================] - 2820s 3s/step - loss: 0.7011 - rpn_class_loss: 0.0105 - rpn_bbox_loss: 0.2113 - mrcnn_class_loss: 0.1638 - mrcnn_bbox_loss: 0.0799 - mrcnn_mask_loss: 0.2356 - val_loss: 1.0790 - val_rpn_class_loss: 0.0180 - val_rpn_bbox_loss: 0.2843 - val_mrcnn_class_loss: 0.3451 - val_mrcnn_bbox_loss: 0

KeyboardInterrupt: 